In [3]:
!pip install openpyxl

In [4]:
import pandas as pd
import os

# Load the Excel file
file_path = "C:/Users/sijan/Desktop/mini_project.xlsx"
excel_data = pd.ExcelFile(file_path)

# Parse the 'NEPSE Index by Sector' sheet, starting from the second row for the header
nepse_index_df = excel_data.parse('NEPSE Index', header=0, skiprows=1)

# Parse the 'Market capitalization' sheet, starting from the third row for the header
market_cap_df = excel_data.parse('Market capitalization', header=0, skiprows=2)

# Drop the last column of market_cap_df
market_cap_df.drop(columns=market_cap_df.columns[-1], inplace=True)

# Function to rename columns with special case handling
def rename_columns(df, prefix):
    new_columns = {}
    for col in df.columns:
        if col != 'Date/Month':
            if "Insurance/" in col:
                initials = "I"
            elif "Non-Life" in col:
                initials = "NL"
            elif col == "Hydro":
                initials = "Hydro"
            else:
                parts = col.split()
                initials = ''.join(part[0] for part in parts if part.isalnum())
            # Ensure unique initials to avoid conflicts like M_H
            counter = 1
            original_initials = initials
            while f"{prefix}_{initials}" in new_columns.values():
                initials = f"{original_initials}{counter}"
                counter += 1
            new_columns[col] = f"{prefix}_{initials}"
        else:
            new_columns[col] = f"{prefix}_Date/Month"
    df.rename(columns=new_columns, inplace=True)

# Rename columns in both dataframes
rename_columns(nepse_index_df, 'N')
rename_columns(market_cap_df, 'M')

# Fill empty cells with 0
nepse_index_df.fillna(0, inplace=True)
market_cap_df.fillna(0, inplace=True)

# Ensure the 'Date/Month' column is properly formatted to avoid displaying ###
nepse_index_df['N_Date/Month'] = pd.to_datetime(nepse_index_df['N_Date/Month'], format='%d-%b-%y', errors='coerce')
market_cap_df['M_Date/Month'] = pd.to_datetime(market_cap_df['M_Date/Month'], format='%d-%b-%y', errors='coerce')

# Merge the two dataframes on 'Date/Month'
merged_df = pd.merge(nepse_index_df, market_cap_df, left_on='N_Date/Month', right_on='M_Date/Month', how='outer')

# Drop the second 'Date/Month' column
merged_df.drop(columns=['M_Date/Month'], inplace=True)

# Rename the remaining 'Date/Month' column back to 'Date/Month'
merged_df.rename(columns={'N_Date/Month': 'Date/Month'}, inplace=True)

# Rearrange columns to keep 'Date/Month' at the beginning
columns = ['Date/Month'] + [col for col in merged_df.columns if col != 'Date/Month']
merged_df = merged_df[columns]

# Convert dates back to the desired format for output
merged_df['Date/Month'] = merged_df['Date/Month'].dt.strftime('%d-%b-%y')

# Drop the 'N_1' column
merged_df.drop(columns=['N_1'], inplace=True, errors='ignore')

# Save the merged dataframe to the specified path
desktop_path = "C:/Users/sijan/Desktop"
output_file_path = os.path.join(desktop_path, "merged_nepse_market_cap.xlsx")

# Save the merged dataframe to the new path
merged_df.to_excel(output_file_path, index=False)

print(f"File saved to: {output_file_path}")




File saved to: C:/Users/sijan/Desktop\merged_nepse_market_cap.xlsx
